In [1]:
talk_model_name="dicta-il/dictalm-7b-instruct"

In [15]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM

In [5]:
model=AutoModelForCausalLM.from_pretrained(talk_model_name,trust_remote_code=True)
tokenizer=AutoTokenizer.from_pretrained(talk_model_name,trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# main code

In [14]:
model.to('cuda')

MegatronGPTForCausalLM(
  (megatron_gpt): MegatronGPTModel(
    (embed_in): Embedding(56064, 4096)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x MegatronGPTLayer(
        (input_layernorm): MegatronGPTLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): MegatronGPTLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (self_attention): MegatronGPTAttention(
          (rotary_emb): MegatronGPTRotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MegatronGPTMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=10880, bias=True)
          (dense_4h_to_h): 

In [24]:
def generate_prompt(text, question):
    """
    Generates a prompt for the model to encourage direct quoting from the text in response to a question.

    Args:
    - text (str): The Hebrew text to be summarized.
    - question (str): The user's specific question.

    Returns:
    - str: The generated prompt for the model.
    """

    prompt = f"""
    טקסט: {text}
    שאלה: {question}
    הנחיות: נא לצטט באופן ישיר מהטקסט כחלק מהתשובה. התשובה צריכה להתייחס במדויק לפרטים המופיעים בטקסט.
    תשובה:
    """

    return prompt


In [25]:
@torch.no_grad
def generate_summary(text, question,model=model, tokenizer=tokenizer):
    """
    Generates a summary of a given Hebrew text from the Supreme Court decisions database,
    focusing on aspects relevant to the user's question.

    Args:
    - model (transformers.PreTrainedModel): The loaded Hugging Face model.
    - tokenizer (transformers.PreTrainedTokenizer): The loaded tokenizer.
    - text (str): The Hebrew text to summarize.
    - question (str): The user's specific question to focus the summary on.

    Returns:
    - str: The generated summary.
    """

    # Combine the text and the question to provide context to the model
    prompt = generate_prompt(text,question)#f"טקסט: {text}\nשאלה: {question}\nתקציר:"

    # Encode the combined prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)

    # Generate the response
    output = model.generate(input_ids, max_new_tokens=150, num_return_sequences=1, no_repeat_ngram_size=2)

    # Decode the generated text
    summary = tokenizer.decode(output[0][input_ids.shape[-1]:].cpu(), skip_special_tokens=True)

    return summary


# test

In [26]:
text="כשהייתי בן שש, ראיתי פעם בספר על יער ויראני שכותרתו הייתה 'סיפורים מחיי החיות' תמונה מרהיבה. תיארה את נחש בולע חיה. הנה העתק של הציור. כתוב בספר: 'נחשי הבואה בולעים את טרפם כולו, בלי ללעוס אותו. אחר כך אינם יכולים לנוע והם ישנים לאורך ששת החודשים שלוקח להם לעכל.'"
question="מה ראה הנסיך הקטן כשהיה בן שש שהשפיע עליו כל כך?"

In [27]:
generate_summary(text,question)

' : "נחשים בולעי חיה"'

In [28]:
generate_prompt("1","2")

'\n    טקסט: 1\n    שאלה: 2\n    הנחיות: נא לצטט באופן ישיר מהטקסט כחלק מהתשובה. התשובה צריכה להתייחס במדויק לפרטים המופיעים בטקסט.\n    תשובה:\n    '